<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1> 

<h3 align=left><font size = 5>1. Import libraries</font></h3> 

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# from bs4 import BeautifulSoup
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests # Requests allows you to send organic, grass-fed HTTP/1.1 requests, without the need for manual labor.

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [ ]:
!conda install lxml --yes

In [ ]:
!conda update -c conda-forge -y pandas

In [ ]:
conda install -c anaconda beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import pandas as pd

<h2 align=left><font size = 5>2. Scraping table from Wikipedia page </font></h2> 
<h3 align=left><font size = 3>Use the BeautifulSoup package or <b>any other way</b> you are comfortable with </font></h3> 
<h3 align=left><font size = 3>I am using pd.read_html </font></h3> 



In [ ]:
df = pd.read_html(r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',match= 'Neighborhood')
df = df[0]
df = pd.DataFrame(df) 
df.head()

<h3 align=left><font size = 3> - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood <br>
- Ignore cells with a borough that is Not assigned. </font></h3> 

In [ ]:
df.columns= ["PostalCode","Borough","Neighborhood"] # The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [ ]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True) # Ignore cells with a borough that is Not assigned.

In [ ]:
df.head()

<h3 align=left><font size = 3> Merge cells with same postal code. <br>
More than one neighborhood can exist in one postal code area, separated them with ","  </font></h3> 

In [ ]:
df=df.groupby(['PostalCode','Borough'],as_index=False).agg(lambda x: ','.join(x)) #Merge cells with same postal code.
#More than one neighborhood can exist in one postal code area, separated them with ","
df["Neighborhood"]= df["Neighborhood"].str.replace(" / ", ", ")
df.head()

<h3 align=left><font size = 3> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  </font></h3> 

In [ ]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
e = len(df)
i = 1
while i < e:
    y = df.iat[i,2]
    
    if y == 'Not assigned':
        df.iat[i,2] = df.iat[i,1]
    i += 1
df.head()
# See Queen's Park (last row) for example

<h3 align=left><font size = 3> Show final result - with 15 random rows</font></h3> 

In [ ]:
df_random = df.sample(n=15)
df_random

<h3 align=left><font size = 3> Use the .shape method to print the number of rows of your dataframe.  <br>
Total rows: 103</font></h3> 

In [ ]:
df.shape[0]

<h3 align=left><font size = 3> Save Dataframe to CSV file</font></h3> 


In [ ]:
df.to_csv(r'Toronto.csv', index = False)

In [ ]:
coordinate = pd.read_csv('Geospatial_Coordinates.csv') # downloaded file from https://cocl.us/Geospatial_data
df=df.merge(coordinate,how='left',left_on='PostalCode',right_on='Postal Code')
del df['Postal Code'] # get rid of additional column
df